### Importando Librerias Necesarias

In [52]:
import pandas as pd

### leyendo del dataset las hojas HECHOS y VICTIMAS

In [53]:
#tuve que instalar openpyxl para leer los archivos
hechos = pd.read_excel('../Datasets/homicidios.xlsx', sheet_name='HECHOS')
victimas =  pd.read_excel('../Datasets/homicidios.xlsx', sheet_name='VICTIMAS') 

In [54]:
hechos.columns

Index(['ID', 'N_VICTIMAS', 'FECHA', 'AAAA', 'MM', 'DD', 'HORA', 'HH',
       'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE', 'Calle', 'Altura', 'Cruce',
       'Dirección Normalizada', 'COMUNA', 'XY (CABA)', 'pos x', 'pos y',
       'PARTICIPANTES', 'VICTIMA', 'ACUSADO'],
      dtype='object')

#### lo primero seria obtener las columnas a utilizar para el analisis

In [55]:
hechos.drop(columns=['DD', 'HORA', 'LUGAR_DEL_HECHO', 'Calle', 'Altura', 'Cruce', 'Dirección Normalizada', 'XY (CABA)', 'PARTICIPANTES'], inplace=True)

In [56]:
hechos.head(5)

,ID,N_VICTIMAS,FECHA,AAAA,MM,HH,TIPO_DE_CALLE,COMUNA,pos x,pos y,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,4,AVENIDA,8,-58.47533969,-34.68757022,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,1,GRAL PAZ,9,-58.50877521,-34.66977709,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,7,AVENIDA,1,-58.39040293,-34.63189362,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,0,AVENIDA,8,-58.46503904,-34.68092974,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,5,AVENIDA,1,-58.38718297,-34.62246630,MOTO,PASAJEROS


In [57]:
hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   ID             696 non-null    object        
 1   N_VICTIMAS     696 non-null    int64         
 2   FECHA          696 non-null    datetime64[ns]
 3   AAAA           696 non-null    int64         
 4   MM             696 non-null    int64         
 5   HH             696 non-null    object        
 6   TIPO_DE_CALLE  696 non-null    object        
 7   COMUNA         696 non-null    int64         
 8   pos x          696 non-null    object        
 9   pos y          696 non-null    object        
 10  VICTIMA        696 non-null    object        
 11  ACUSADO        696 non-null    object        
dtypes: datetime64[ns](1), int64(4), object(7)
memory usage: 65.4+ KB


#### lo siguiente seria verificar elementos faltantes para este Dataset seria elementos iguales a 'SD'

In [58]:
hechos.apply(lambda x: x.map(lambda y: 1 if y == 'SD' else 0)).sum()

ID                0
N_VICTIMAS        0
FECHA             0
AAAA              0
MM                0
HH                1
TIPO_DE_CALLE     0
COMUNA            0
pos x             0
pos y             0
VICTIMA           9
ACUSADO          23
dtype: int64

puede no tener datos sobre el medio de transporte de la victima o acusado, pero si verificar la franja horaria, donde vemos que no tenemos registro del horario del percance

#### Continuamos con el siguiente

In [59]:
victimas.columns

Index(['ID_hecho', 'FECHA', 'AAAA', 'MM', 'DD', 'ROL', 'VICTIMA', 'SEXO',
       'EDAD', 'FECHA_FALLECIMIENTO'],
      dtype='object')

In [60]:
victimas.drop(columns=['FECHA', 'AAAA', 'MM', 'DD', 'VICTIMA'], inplace=True)

In [61]:
victimas.head()

,ID_hecho,ROL,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,CONDUCTOR,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,CONDUCTOR,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,CONDUCTOR,MASCULINO,30,2016-01-03 00:00:00
3,2016-0004,CONDUCTOR,MASCULINO,18,SD
4,2016-0005,CONDUCTOR,MASCULINO,29,2016-02-01 00:00:00


#### Nos interesa identificar los victimas fatales por lo que creamos una columna

In [62]:
victimas['FATAL'] = victimas['FECHA_FALLECIMIENTO'].apply(lambda y: 0 if y == 'SD' else 1)

In [63]:
victimas.head()

,ID_hecho,ROL,SEXO,EDAD,FECHA_FALLECIMIENTO,FATAL
0,2016-0001,CONDUCTOR,MASCULINO,19,2016-01-01 00:00:00,1
1,2016-0002,CONDUCTOR,MASCULINO,70,2016-01-02 00:00:00,1
2,2016-0003,CONDUCTOR,MASCULINO,30,2016-01-03 00:00:00,1
3,2016-0004,CONDUCTOR,MASCULINO,18,SD,0
4,2016-0005,CONDUCTOR,MASCULINO,29,2016-02-01 00:00:00,1


In [64]:
victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID_hecho             717 non-null    object
 1   ROL                  717 non-null    object
 2   SEXO                 717 non-null    object
 3   EDAD                 717 non-null    object
 4   FECHA_FALLECIMIENTO  717 non-null    object
 5   FATAL                717 non-null    int64 
dtypes: int64(1), object(5)
memory usage: 33.7+ KB


In [65]:
fatal = victimas.groupby('ID_hecho')['FATAL'].sum().reset_index()

In [66]:
fatal.rename(columns={'ID_hecho':'ID'}, inplace=True)

In [67]:
fatal.head()

,ID,FATAL
0,2016-0001,1
1,2016-0002,1
2,2016-0003,1
3,2016-0004,0
4,2016-0005,1


In [68]:
hechos = pd.merge(hechos,fatal, on='ID',how='inner')

In [69]:
hechos

,ID,N_VICTIMAS,FECHA,AAAA,MM,HH,TIPO_DE_CALLE,COMUNA,pos x,pos y,VICTIMA,ACUSADO,FATAL
0,2016-0001,1,2016-01-01,2016,1,4,AVENIDA,8,-58.47533969,-34.68757022,MOTO,AUTO,1
1,2016-0002,1,2016-01-02,2016,1,1,GRAL PAZ,9,-58.50877521,-34.66977709,AUTO,PASAJEROS,1
2,2016-0003,1,2016-01-03,2016,1,7,AVENIDA,1,-58.39040293,-34.63189362,MOTO,AUTO,1
3,2016-0004,1,2016-01-10,2016,1,0,AVENIDA,8,-58.46503904,-34.68092974,MOTO,SD,0
4,2016-0005,1,2016-01-21,2016,1,5,AVENIDA,1,-58.38718297,-34.62246630,MOTO,PASAJEROS,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
691,2021-0093,1,2021-12-13,2021,12,17,AVENIDA,7,-58.43353773,-34.64561636,MOTO,AUTO,1
692,2021-0094,1,2021-12-20,2021,12,1,AUTOPISTA,9,-58.46739825,-34.65117757,MOTO,AUTO,1
693,2021-0095,1,2021-12-30,2021,12,0,AVENIDA,11,-58.47293407,-34.61984745,MOTO,CARGAS,1
694,2021-0096,1,2021-12-15,2021,12,10,AVENIDA,9,-58.47066794,-34.65021673,AUTO,CARGAS,1


#### No todos tienen ubicacion por lo que veremos el DS original para ver que podemos hacer

In [70]:
hechos[hechos['pos x']=='.'][['ID','FATAL']]

,ID,FATAL
38,2016-0052,0
106,2016-0136,0
119,2016-0151,0
139,2016-0174,0
176,2017-0042,1
180,2017-0050,2
181,2017-0051,1
256,2017-0140,1
313,2018-0039,1
546,2020-0026,1


dejamos en nulo las ubicaciones sin dato para cuando querramos graficar

In [71]:
h = pd.read_excel('../Datasets/homicidios.xlsx', sheet_name='HECHOS')

In [72]:
pd.merge(h,hechos[hechos['pos x']=='.'][['ID','FATAL']], on='ID', how='inner')

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO,FATAL
0,2016-0052,1,2016-04-20,2016,4,20,20:00:00,20,AUTOPISTA LUGONES PK 10000,AUTOPISTA,...,NaN,NaN,13,Point (. .),.,.,MOTO-SD,MOTO,SD,0
1,2016-0136,1,2016-10-25,2016,10,25,00:00:00,0,AU BUENOS AIRES - LA PLATA KM. 4,AUTOPISTA,...,NaN,NaN,4,Point (. .),.,.,MOTO-CARGAS,MOTO,CARGAS,0
2,2016-0151,1,2016-11-18,2016,11,18,20:35:00,20,SD,CALLE,...,NaN,NaN,0,Point (. .),.,.,PEATON-SD,PEATON,SD,0
3,2016-0174,1,2016-12-27,2016,12,27,00:00:00,0,AUTOPISTA 25 DE MAYO,AUTOPISTA,...,NaN,AUTOPISTA 25 DE MAYO,0,Point (. .),.,.,SD-SD,SD,SD,0
4,2017-0042,1,2017-04-10,2017,4,10,09:00:00,9,AV. LEOPOLDO LUGONES PKM 6900,GRAL PAZ,...,NaN,"LUGONES, LEOPOLDO AV.",14,Point (. .),.,.,MOTO-CARGAS,MOTO,CARGAS,1
5,2017-0050,2,2017-04-28,2017,4,28,11:08:08,11,AU PERITO MORENO Y RAMAL ENLACE AU1/AU6,AUTOPISTA,...,NaN,NaN,9,Point (. .),.,.,MOTO-CARGAS,MOTO,CARGAS,2
6,2017-0051,1,2017-05-01,2017,5,1,03:47:47,3,AU DELLEPIANE 2400,AUTOPISTA,...,NaN,NaN,7,Point (. .),.,.,AUTO-AUTO,AUTO,AUTO,1
7,2017-0140,1,2017-11-19,2017,11,19,23:22:17,23,AU ARTURO FRONDIZI PKM 3100,AUTOPISTA,...,NaN,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,4,Point (. .),.,.,MOTO-PASAJEROS,MOTO,PASAJEROS,1
8,2018-0039,1,2018-04-21,2018,4,21,22:15:00,22,AUTOPISTA LUGONES KM 4.7,AUTOPISTA,...,NaN,NaN,14,Point (. .),.,.,PEATON-AUTO,PEATON,AUTO,1
9,2020-0026,1,2020-05-17,2020,5,17,06:40:00,6,"LUGONES, LEOPOLDO AV. KM 6,1",AUTOPISTA,...,NaN,NaN,14,Point (. .),.,.,MOTO-OBJETO FIJO,MOTO,OBJETO FIJO,1


In [73]:
hechos.loc[hechos[hechos['pos x']=='.'].index, ['pos x','pos y']]=None

In [74]:
hechos[hechos['pos x'].isna()]

,ID,N_VICTIMAS,FECHA,AAAA,MM,HH,TIPO_DE_CALLE,COMUNA,pos x,pos y,VICTIMA,ACUSADO,FATAL
38,2016-0052,1,2016-04-20,2016,4,20,AUTOPISTA,13,None,None,MOTO,SD,0
106,2016-0136,1,2016-10-25,2016,10,0,AUTOPISTA,4,None,None,MOTO,CARGAS,0
119,2016-0151,1,2016-11-18,2016,11,20,CALLE,0,None,None,PEATON,SD,0
139,2016-0174,1,2016-12-27,2016,12,0,AUTOPISTA,0,None,None,SD,SD,0
176,2017-0042,1,2017-04-10,2017,4,9,GRAL PAZ,14,None,None,MOTO,CARGAS,1
180,2017-0050,2,2017-04-28,2017,4,11,AUTOPISTA,9,None,None,MOTO,CARGAS,2
181,2017-0051,1,2017-05-01,2017,5,3,AUTOPISTA,7,None,None,AUTO,AUTO,1
256,2017-0140,1,2017-11-19,2017,11,23,AUTOPISTA,4,None,None,MOTO,PASAJEROS,1
313,2018-0039,1,2018-04-21,2018,4,22,AUTOPISTA,14,None,None,PEATON,AUTO,1
546,2020-0026,1,2020-05-17,2020,5,6,AUTOPISTA,14,None,None,MOTO,OBJETO FIJO,1


#### nos quedamos con 2 archivos

In [78]:
hechos.head(5)

,ID,N_VICTIMAS,FECHA,AAAA,MM,HH,TIPO_DE_CALLE,COMUNA,pos x,pos y,VICTIMA,ACUSADO,FATAL
0,2016-0001,1,2016-01-01,2016,1,4,AVENIDA,8,-58.47533969,-34.68757022,MOTO,AUTO,1
1,2016-0002,1,2016-01-02,2016,1,1,GRAL PAZ,9,-58.50877521,-34.66977709,AUTO,PASAJEROS,1
2,2016-0003,1,2016-01-03,2016,1,7,AVENIDA,1,-58.39040293,-34.63189362,MOTO,AUTO,1
3,2016-0004,1,2016-01-10,2016,1,0,AVENIDA,8,-58.46503904,-34.68092974,MOTO,SD,0
4,2016-0005,1,2016-01-21,2016,1,5,AVENIDA,1,-58.38718297,-34.62246630,MOTO,PASAJEROS,1


este archivo contendra datos del siniestro con el dato si este fue o no fatal

In [79]:
victimas.head(5)

,ID_hecho,ROL,SEXO,EDAD,FECHA_FALLECIMIENTO,FATAL
0,2016-0001,CONDUCTOR,MASCULINO,19,2016-01-01 00:00:00,1
1,2016-0002,CONDUCTOR,MASCULINO,70,2016-01-02 00:00:00,1
2,2016-0003,CONDUCTOR,MASCULINO,30,2016-01-03 00:00:00,1
3,2016-0004,CONDUCTOR,MASCULINO,18,SD,0
4,2016-0005,CONDUCTOR,MASCULINO,29,2016-02-01 00:00:00,1


este archivo contendra datos de la victima

In [76]:
hechos.to_csv('../Datasets/hechos.csv', index=False)

In [77]:
victimas.to_csv('../Datasets/victimas.csv', index=False)